In [22]:
# Se descargan librerías necesarias para trabajar
import pandas as pd
import sqlite3 as sq
import warnings

# Definir el path del archivo
path = './DB/homicidios.xlsx'

# Se eliminan warns
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [23]:
# Cargar los datos
dataHechos = pd.read_excel(path, sheet_name='HECHOS')
dataVictimas = pd.read_excel(path, sheet_name='VICTIMAS')

In [24]:
# Se genera una conexión local de BD
conex = sq.connect('pbi.db')

In [14]:
# Convertir la columna 'HORA' a tipo datetime
dataHechos['HORA'] = pd.to_datetime(dataHechos['HORA'], format='%H:%M:%S', errors='coerce')
dataHechos['HORA'].fillna(pd.Timestamp('00:00:00'), inplace=True)

# Convertir la columna 'FECHA' a tipo datetime
dataHechos['FECHA'] = pd.to_datetime(dataHechos['FECHA'], format='%d/%m/%Y', errors='coerce')

# Convertir la columna 'EDAD' a numérico
dataVictimas['EDAD'] = pd.to_numeric(dataVictimas['EDAD'], errors='coerce')
dataVictimas['EDAD'].fillna(0, inplace=True)

In [15]:
# Eliminar columnas con más del 90% de valores faltantes
umbral_nulos = 0.9
porcentaje_nulos_hechos = dataHechos.isnull().mean()
columnas_a_eliminar_hechos = porcentaje_nulos_hechos[porcentaje_nulos_hechos > umbral_nulos].index
dataHechos.drop(columns=columnas_a_eliminar_hechos, inplace=True)

In [16]:
# Se manda información a la base de datos local
dataHechos.to_sql('hechos', con=conex, if_exists='replace', index=False)
dataVictimas.to_sql('victimas', con=conex, index=False, if_exists='replace')

696

In [20]:
# Se genera una sola tabla con un join, entre las tablas de hechos y victimas
query = """
SELECT 
    H.ID, H.N_VICTIMAS, H.FECHA AS HECHO_FECHA, H.AAAA, H.MM, H.DD, H.HORA, H.HH,
    H.LUGAR_DEL_HECHO, H.TIPO_DE_CALLE, H.Calle, H.Cruce, H.`Dirección Normalizada`,
    H.COMUNA, H.`XY (CABA)`, H.`pos x`, H.`pos y`, H.PARTICIPANTES, H.VICTIMA AS HECHO_VICTIMA, H.ACUSADO,
    V.FECHA AS VICTIMA_FECHA, V.ROL, V.VICTIMA AS VICTIMA_VICTIMA, V.SEXO, V.EDAD, V.FECHA_FALLECIMIENTO
FROM hechos H
LEFT JOIN victimas V ON H.ID = V.ID_hecho
"""
x = pd.read_sql_query(query,con=conex)
x.to_sql('JOIN', con=conex,if_exists='replace', index=False)

717

In [21]:
x.to_excel('DB/PBI.xlsx', index=False)